# Rating of a Plain Fin-Tube Counterflow Air Condenser
---

**Importing libraries and notebook preparation**

In [1]:
import warnings
from hvac import Quantity
from hvac.fluids import HumidAir, Fluid, CoolPropWarning
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AC

warnings.filterwarnings('ignore', category=CoolPropWarning)
Q_ = Quantity

## Refrigerant

In [2]:
R134a = Fluid('R134a')

## Condenser Model

Type of condenser: plain fin-tube, counter-flow, air condenser.

In [3]:
Condenser = PFT_CF_AC

To create an instance of the condenser model, we need to specify the width and height of the condenser's frontal area, the number of rows, the geometrical properties of the heat exchanger core, and the thermal conductivity of the fins. The dimensions of the frontal area have been determined in our preliminary calculations (see *ex01_prelim.ipynb*).

In [4]:
condenser = Condenser(
    W_fro=Q_(1.003, 'm'),
    H_fro=Q_(0.334, 'm'),
    N_rows=5,  # <-- can be modified until a solution is found
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(8.422, 'mm'),
    D_ext=Q_(10.2, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm'),
    k_fin=Q_(237, 'W / (m * K)')
)

## Condenser Rating

On the condenser model, we set the known operating conditions:
- the air mass flow rate through the condenser
- the refrigerant mass flow rate
- the state of air entering the condenser
- the state of refrigerant entering the condenser

The result of the condenser rating will give us:
- the state of refrigerant leaving the condenser
- the state of air leaving the condenser
- the heat rejection rate from the refrigerant to the air in the condenser
- the degree of refrigerant subcooling
- the air-side pressure drop across the condenser

When we rated the evaporator (see notebook *ex02_air_evaporator.ipynb*) to get at the desired temperature (13 °C) of air leaving the evaporator, we've noticed that the mass flow rate of refrigerant needed to superheat the refrigerant (10 K) was more than the mass flow rate we had determined in our preliminary calculations (see notebook *ex01_prelim_calc.ipynb*): 142.631 kg/h instead of 136.023 kg/h. This was because the evaporator extracts some more moisture from the air than we had assumed in our preliminary calculations, where we arbitrarily assumed that air leaving the evaporator would be nearly saturated (RH 90 %).

To rate the condenser, we need the refrigerant state at the entrance of the condenser (which we assume equal to the state of refrigerant discharged by the compressor). Using the compressor selection program again, we can repeat the compressor selection routine with the heat absorption rate that followed from the evaporator rating (keeping the evaporation temperature, the condensing temperature, and the degree of superheating and subcooling the same as before). This leads us still to a variable speed compressor DANFOSS VTZ038-G running at 4407 rpm and the estimated discharge temperature of the refrigerant is about 80.1 °C. We keep the condensing pressure we determined in the preliminary calculations.

We recalculate the heat rejection rate in the condenser:

In [5]:
# Refrigerant state at condenser inlet:
rfg_in = R134a(T=Q_(80.1, 'degC'), P=Q_(13.179, 'bar'))
# Selected condensing temperature:
T_cnd = Q_(50, 'degC')
rfg_sat = R134a(T=T_cnd, x=Q_(0, 'frac'))
P_cnd = rfg_sat.P
# Selected degree of subcooling:
dT_sc = Q_(5, 'K')
# Refrigerant state at condenser outlet:
T_rfg_out = rfg_sat.T - dT_sc 
rfg_out = R134a(T=T_rfg_out, P=P_cnd)
# Selected mass flow rate of refrigerant
# (which follows from the evaporator rating)
rfg_m_dot = Q_(142.631, 'kg / hr')
# Heat rejection rate of refrigerant in the condenser:
cnd_Q_dot = rfg_m_dot * (rfg_in.h - rfg_out.h)
print(cnd_Q_dot.to('kW'))

7.68514307400804 kilowatt


And we also recalculate the mass flow rate of air through the condenser:

In [6]:
# Air state at condenser inlet:
air_in = HumidAir(Tdb=Q_(35.0, 'degC'), RH=Q_(30, 'pct'))
# Selected air temperature rise across the condenser
# (this choice depends also on the selected condensing temperature):
air_dT = Q_(10, 'K')  
# Air state at the condenser outlet:
air_out = HumidAir(Tdb=air_in.Tdb + air_dT, W=air_in.W)
# Mass flow rate of air through the condenser:
air_m_dot = cnd_Q_dot / (air_out.h - air_in.h)
print(air_m_dot.to('kg/hr'))

2694.2933834355113 kilogram / hour


Now, we can try to rate the condenser. It may happen that the rating routine will raise an error with the operating conditions we've set. In that case, we can try again with a bigger number of tube rows, by altering the operating conditions (e.g., by lowering the allowable temperature rise of the air), by altering the heat exchanger core geometry, or increasing the air-side frontal area surface.

In [7]:
air_out, rfg_out = condenser.solve(
    air_in=air_in,
    air_m_dot=air_m_dot,
    rfg_in=rfg_in,
    rfg_m_dot=rfg_m_dot
)

[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] Iteration 1
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] Try with subcooling flow length 0.100 mm.
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] 1. Subcooling region. Temperature of leaving refrigerant = 49.792 °C (deviation = 7.292 K).
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] 2. Subcooling region. Temperature of leaving refrigerant = 49.789 °C (deviation = -0.003 K).
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] Subcooling region. Leaving air temperature = 35.017 °C.
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser | DEBUG] 1. Condensing region. Flow length = 1.000 mm -> heat transfer rate = 138.175 W (deviation = -5876.654 W).
[17356 | hvac.heat_exchanger.recuperator.fintube.continuous_fin.air_condenser

Final results:

In [10]:
print(
    f"leaving air state = {air_out.Tdb.to('degC'):~P.1f} DB, "
    f"{air_out.RH.to('pct'):~P.0f} RH",
    f"leaving refrigerant temperature = {rfg_out.T.to('degC'):~P.2f}",
    f"leaving refrigerant enthalpy = {rfg_out.h.to('kJ / kg'):~P.2f}",
    f"degree of subcooling = {condenser.dT_sc.to('K'):~P.2f}",
    f"heat rejection rate = {condenser.Q_dot.to('kW'):~P.3f}",
    f"air-side pressure drop = {condenser.air_dP.to('Pa'):~P.3f}",
    sep='\n'
)

leaving air state = 45.0 °C DB, 18 % RH
leaving refrigerant temperature = 45.53 °C
leaving refrigerant enthalpy = 264.71 kJ/kg
degree of subcooling = 4.47 K
heat rejection rate = 7.653 kW
air-side pressure drop = 39.541 Pa


Note: the air-side pressure drop should also be considered when rating the condenser. A large air-side pressure drop would imply we need a larger, more energy consuming fan to push or draw the required air flow through the condenser.

We notice that the state of refrigerant leaving the condenser is not entirely equal to the state we've used when rating the evaporator.

In [12]:
P_evp = R134a(T=Q_(10, 'degC'), x=Q_(1.0, 'frac')).P
evp_rfg_in = R134a(P=P_evp, h=rfg_out.h)
print(
    f"vapor quality of evaporator entering refrigerant = "
    f"{evp_rfg_in.x.to('frac'):~P.2f}"
)

vapor quality of evaporator entering refrigerant = 0.27 frac
